<h1><center>Introduction to Neural Computation Coursework</center></h1>
<h1><center>Group XVI 👁</center></h1>
Jon Duffy, Ed Wong, Shiyu Fan, Yawen Sun

[Click here](https://github.com/JonDuffy/YAD2K) for the github repository.

## Introduction
For this task we look into using a neural network to classify and localise objects in images. We have 14,625 number of preprocessed training images of dimension 400x400 pixels, each of which are paired with a text file that specify classes and, if present, bounding boxes around a present object in that image. These images are from the VOC data set.

We also have 2500 number of testing images of the same format. Our task is to accurately generate an accompanying text file, which specifies the classes and, if present, bounding boxes of all the test images.

Below is the text file for image 2007_000027. Here we can see 20 classes. The “1 0” following the class name indicates that this class is not present in the image. If an object has been detected, the numbers that follow indicate the bounding box with pixel references. The pixels are numbered from top to bottom, then left to right e.g. 1 is pixel (1,1), 2 is pixel (2,1), etc. The numbers are paired such that the first number represents the starting pixel and the second represents how many consecutive pixels after this. This creates a bounding box around our objects.


<code>2007_000027.jpg_aeroplane,1 0
2007_000027.jpg_bicycle,1 0
2007_000027.jpg_bird,1 0
2007_000027.jpg_boat,1 0
2007_000027.jpg_bottle,1 0
2007_000027.jpg_bus,1 0
2007_000027.jpg_car,1 0
2007_000027.jpg_cat,1 0
2007_000027.jpg_chair,1 0
2007_000027.jpg_cow,1 0
2007_000027.jpg_diningtable,1 0
2007_000027.jpg_dog,1 0
2007_000027.jpg_horse,1 0
2007_000027.jpg_motorbike,1 0
2007_000027.jpg_person,32143 144 32543 144 32943 144 33343 144 33743 144 34143 144 34543 144 34943 144 35343 144 35743 144 36143 144 36543 144 36943 144 37343 144 37743 144 38143 144 38543 144 38943 144 39343 144 39743 144 40143 144 40543 144 40943 144 41343 144 41743 144 42143 144 42543 144 42943 144 43343 144 43743 144 44143 144 44543 144 44943 144 45343 144 45743 144 46143 144 46543 144 46943 144 47343 144 47743 144 48143 144 48543 144 48943 144 49343 144 49743 144 50143 144 50543 144 50943 144 51343 144 51743 144 52143 144 52543 144 52943 144 53343 144 53743 144 54143 144 54543 144 54943 144 55343 144 55743 144 56143 144 56543 144 56943 144 57343 144 57743 144 58143 144 58543 144 58943 144 59343 144 59743 144 60143 144 60543 144 60943 144 61343 144 61743 144 62143 144 62543 144 62943 144 63343 144 63743 144 64143 144 64543 144 64943 144 65343 144 65743 144 66143 144 66543 144 66943 144 67343 144 67743 144 68143 144 68543 144 68943 144 69343 144 69743 144 70143 144 70543 144 70943 144 71343 144 71743 144 72143 144 72543 144 72943 144 73343 144 73743 144 74143 144 74543 144 74943 144 75343 144 75743 144 76143 144 76543 144 76943 144 77343 144 77743 144 78143 144 78543 144 78943 144 79343 144 79743 144 80143 144 80543 144 80943 144 81343 144 81743 144 82143 144 82543 144 82943 144 83343 144 83743 144 84143 144 84543 144 84943 144 85343 144 85743 144 86143 144 86543 144 86943 144 87343 144 87743 144 88143 144 88543 144 88943 144 89343 144 89743 144 90143 144 90543 144 90943 144 91343 144 91743 144 92143 144 92543 144 92943 144 93343 144 93743 144 94143 144 94543 144 94943 144 95343 144 95743 144 96143 144 96543 144 96943 144 97343 144 97743 144 98143 144 98543 144 98943 144 99343 144 99743 144 100143 144 100543 144 100943 144 101343 144 101743 144 102143 144 102543 144 102943 144 103343 144 103743 144 104143 144 104543 144 104943 144 105343 144 105743 144 106143 144 106543 144 106943 144 107343 144 107743 144 108143 144 108543 144 108943 144 109343 144 109743 144 110143 144 110543 144 110943 144 111343 144 111743 144
2007_000027.jpg_pottedplant,1 0
2007_000027.jpg_sheep,1 0
2007_000027.jpg_sofa,1 0
2007_000027.jpg_train,1 0
2007_000027.jpg_tvmonitor,1 0
</code>

This text file corresponds to this image:

<img src="files/figures/2007_000027.jpg">

Object classification and localisation is an active area of research within machine learning. The subset of the VOC dataset that we have here is a standard benchmark to measure the performance of neural networks.

We aim to experiment with a wide range of features of the network such as hyperparameters and functions within the network. We aim to make changes to improve the classification and localization of the neural network as well as understanding the impact of these changes on this classification and localization problem. The quality of the neural network will be measured by not only our score on kaggle, but cross validation.

We aim to improve the accuracy of our neural network by training and experimenting with different parameters.

## Design
### Initial Approach
Our initial approach was to create a neural network from scratch so that we could experiment with all of the details of the network, such as the topology, applying different types of layers (such as fully connected layers, convolutional layers), learning rates etc. However, there were a few disadvantages/bottlenecks that we came across:
<ul>
    <li>Approaching this problem by working from the ground up leaves much <b>more room for error</b>.
    <li>Implementing a network from scratch means that we must train all of our weights and biases using many epochs of our training set. With object detection and classification, it is generally accepted that deep neural networks are better at this task than shallow neural networks. However, deep neural networks have many more parameters to calculate and iterate. As we have 14,625 training images, this presents a <b>computational resource bottleneck</b>. Even if we were to utilise cloud based services such a google cloud, or amazon web services, we will still run into another barrier.
    <li>Time is a crucial and precious factor to consider. According to [Yang You, et al.](https://arxiv.org/pdf/1709.05011.pdf), training a network can take weeks.
</ul>

As a result we decided to change our approach to take an existing network with pretrained weights, and update these weights by training with our dataset.
### Researching Existing Networks
When researching existing Neural Networks, we came across many different variations. For instance, R-CNN and SSD300 to name a couple. However, a neural network that caught our eye was [YOLOv2](https://pjreddie.com/darknet/yolo/), because of [this video](https://www.youtube.com/watch?v=VOC3huqHrss) demonstration that applied this neural network to a movie clip in real time. This video clip showcased the speed of this network- a factor which was crucial.

After some further research into this neural network, we analysed the topology of it. It consisted of 24 convolutional layers followed by 2 fully connected layers. We could experiment with the topology of the network but adding layers, removing layers, changing the dimensions of the layers etc. However, due to time constraints, we instead focus on fixing this topology and altering other features, ie the hyperparameters of the network.

Pretrained weights from ImageNet were used on the convolutional layers. This essentially means that they only need to train the weights and biases in the final two fully connected layers, which saves time and resources. We could also implement a similar approach by training an implementation of this network on our training data with pretrained weights on the convolutional layers and we find the weights and biases on the fully connected layer. However due to time limitations, it is more logical to take the YOLOv2 network and train it in the network’s current state to update the current weights by training it on few epochs with our own training set.

Let us briefly touch upon how this network is used in order to both classify and localise objects in an image. First, the image is divided into an equal sized grid, e.g. 9x9. Inside each grid cell, the network assigns several anchor boxes. These anchor boxes then attempt to locate an object. The reason for multiple anchor boxes per cell is so we can classify multiple objects in a single cell (Clearly, anchor boxes limit the number of objects in the same grid cell). After this, it implements a certain loss function that balances the trade off between location error and classification error. This loss value is then used to implement back propagation. The layers of the network consist of convolutional layers (which extract the features) and end with fully connected layers (which calculate the probabilities). The output of the network is a high dimensional tensor with each component of the tensor referring to the grid cell, the boxes, the class and the probability of that class occuring.

This information was found by exploring the network, and from [this paper](https://arxiv.org/abs/1506.02640).

We found a python implementation of YOLOv2 called [YAD2K](https://github.com/allanzelener/YAD2K). This uses the YOLOv2 network design and also Keras, tensorflow, pillow and numpy.
### How we will Train:
The training cycle worked through 3 different stages. Each stage split the data into 1800 training images and 200 validation images:
<ol>
    <li>The first stage was 5 epochs, where the last layer is finetuned.
    <li>The second stage of 30 epochs tunes all the layers. It needs not save weights at end of each epoch, so runs without early stopping and checkpointing.
    <li>The third stage of 30 epochs runs with checkpoint and early stopping to stop as soon as validation loss starts increasing.
</ol>
We seperate whole set of images into 8 subsets. Each of these subset contains 2000 images and train on 2000 images at a time.
### Our Version Of This Network
We aim to change the following features and analyse how they effect the networks ability to detect and classify:
<ul>
    <li>Activation functions
    <li>Anchor box coordinates
    <li>Learning rate
</ul>

## Implementation
### Parser for File Types and Data Format
In order to implement YAD2K, our first task was to write a parser so we can understand the annotation files. Then, we needed to modify the output so the network generates the specific format we require.

However, the data that we have actually has some room for interpretation in some cases. For example, if there are many overlapping objects of the same class, the data we have means that we cannot explicitly decipher between two objects in certain circumstances. We can visualise our data as below. We state three examples where this becomes an issue:
<img src="files/figures/figure3_1.png">
Top left: Original

Top right: We do not know if there is an object inside the “silhouette”

Bottom: We do not know how fair this box extends to.

It would be rare that this situation will indeed occur, however it should still be noted when converting the data format.

The data format that we are converting to and from is much more efficient and robust than the data format we have. It simply states a set of coordinates and dimensions per object in an image. For example, we can express the example we had in the introduction as:

<code>2007_000027.jpg_person, 143 80 287 279
</code>

Where each integer refers to (x_1, y_1, x_2, y_2) which is the bounding box around an object of that class in that image.

We needed to convert our training text files to the appropriate format via the following code. It outputs a numpy file containing two arrays. One of which is image binary data and the other is the annotation data where the $i^{th}$ item in the second array contains the annotations for the $i^{th}$ image in array one.

In [ ]:
#Convert.py
import numpy
import re
import os
import sys
import PIL.Image
import argparse

'''
Builds a numpy array from annotation and images that can be processes by YAD2k
'''

classes_dict = {"aeroplane":0, "bicycle":1, "bird":2, "boat":3, "bottle":4, "bus":5, "car":6, "cat":7, "chair":8, "cow":9, "diningtable":10, "dog":11, "horse":12, "motorbike":13, "person":14, "pottedplant":15, "sheep":16, "sofa":17, "train":18, "tvmonitor":19}

def convert_annotation(image_id):
    '''
    Reads annotations from a file and returns then a numpy array of numpy arrays
    A nested array for each bountding box

    Example
    [[class, x_1, y_1, x_2, y_2]]
    '''
    cleaned_boxes = []
    
    file = open('train/{}'.format(image_id))
 
    lines = file.readlines()
    for line in lines:
        blob , i = str(line).split(".jpg_")
        ''' discard blob, is not needed'''
        parts = i.partition(',')
    
        if parts[2] not in "1 0\n": 
     
            strings = parts[2].split(" ")
  
            nums = [] 
            for item in strings:
                #if len(item) is not 0:
                s = re.sub("[\s+]", "", item)
                s = re.sub(" ", "", item)
                s = re.sub("  ", "", item)
                s = re.sub("\n", "", item)
                try: 

                    nums.append(int(s.strip()))
                except Exception as e:
                    continue
                    # print(e)
                    # print("empty string {}".format(s))


            nums_paried = []


            for i in range((len(nums) //2)):
                nums_paried.append([nums[i * 2] , nums[i* 2 +1]])

            y_1 = [1]
            x_1 = [1]
            y_2 = [1]
            x_2 = [1]


            boxes = {}

            for pair in nums_paried:
           
                if pair[1] in boxes:
                    boxes[pair[1]].append(pair)
                else:
                    boxes[pair[1]] = []
                    boxes[pair[1]].append(pair)


            for key, value in boxes.items():
                y_1 = value[0][0] // 400
                y_2 = value[len(value) - 2][0] // 400
                x_1 = value[0][0] % 400
                x_2 = x_1 + value[0][1]

                #print("class {}, x_1 {}, x_2 {}, y_1 {}, y_2 {}".format(classes_dict[parts[0]],x_1,x_2,y_1,y_2))
                box = [classes_dict[parts[0]], x_1, y_1, y_2 - y_1, x_2 - x_1]
                box = numpy.array(box)
                
                cleaned_boxes.append(box)
    
    return numpy.array(cleaned_boxes)

def generate_set(begin, end, output_file):

    files = os.listdir("train/")

    files.sort()

    text_files = []
    image_files = []


    for i in range(begin, int(end)):
        if ".jpg" in files[i]:
            image_files.append(files[i])
        else:
            text_files.append(files[i])
    '''
    Check that the image and annotation files match

    '''
    assert len(image_files), len(text_files)

    for i in range(len(image_files)):
        image_name = re.sub('.jpg', '',image_files[i])
        file_name = re.sub('.txt','',text_files[i])

        if image_name not in file_name:
            print("annotation: {} and image file: {} does not match".format(file_name, image_name))

    annotations = []

    print(len(image_files), len(text_files))

    for file in text_files:
        annotations.append(convert_annotation(file))

    image_labels = numpy.array(annotations)


    images = []

    for image in image_files:
        img = numpy.array(PIL.Image.open(os.path.join('train', image )).resize((640, 480)).convert('RGB'), dtype=numpy.uint8)
        images.append(img)

    images = numpy.array(images, dtype=numpy.uint8)


    print("Dataset contains {} images".format(images.shape[0]))
    numpy.savez(os.path.join('numpy_arrays', output_file ), images=images, boxes=image_labels)

'''
Set variables to determine which files to collect build into numpy array
'''
parser = argparse.ArgumentParser(description='Generate a training set')
parser.add_argument('--begin', metavar='-b', type=int, help='pick an even number')
parser.add_argument('--end', metavar='-e', type=int, help='Pick an even number')
parser.add_argument('--name', metavar='-n', help='File Name')
args = parser.parse_args()

generate_set(args.begin, args.end, args.name)

We can pinpoint where we lose information in our code for test_yolo.py:

In [4]:
#! /usr/bin/env python
"""Run a YOLO_v2 style detection model on test images."""
import argparse
import colorsys
import imghdr
import os
import random

import numpy as np
from keras import backend as K
from keras.models import load_model
from PIL import Image, ImageDraw, ImageFont

from yad2k.models.keras_yolo import yolo_eval, yolo_head

parser = argparse.ArgumentParser(
    description='Run a YOLO_v2 style detection model on test images..')
parser.add_argument(
    'model_path',
    help='path to h5 model file containing body'
    'of a YOLO_v2 model')
parser.add_argument(
    '-a',
    '--anchors_path',
    help='path to anchors file, defaults to yolo_anchors.txt',
    default='model_data/yolo_anchors.txt')
parser.add_argument(
    '-c',
    '--classes_path',
    help='path to classes file, defaults to coco_classes.txt',
    default='model_data/coco_classes.txt')
parser.add_argument(
    '-t',
    '--test_path',
    help='path to directory of test images, defaults to images/',
    default='images')
parser.add_argument(
    '-o',
    '--output_path',
    help='path to output test images, defaults to images/out',
    default='images/out')
parser.add_argument(
    '-s',
    '--score_threshold',
    type=float,
    help='threshold for bounding box scores, default .3',
    default=.3)
parser.add_argument(
    '-iou',
    '--iou_threshold',
    type=float,
    help='threshold for non max suppression IOU, default .5',
    default=.5)

def get_pixels_for_each_class(image_boxes, image):
        """
        Takes all the box co-ordinates for an image
        returns every pixel value keyed by class
        """

        classes = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog" ,"horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

        pixel_classes = {}

        for class_name in classes:

            all_pixels_in_class = []

            for box in image_boxes:

                if class_name in box[0]:
                    y_1 = box[1][1]
                    y_2 = box[2][1]
                    x_1 = box[1][0]
                    x_2 = box[2][0]
                    for j in range(y_1, y_2):
                        
                        for i in range(x_1, x_2):
                            all_pixels_in_class.append(j * image.width + i)


                all_pixels_in_class = list(set(all_pixels_in_class)) #HERE is where we lose information
                all_pixels_in_class.sort()

            pixel_classes[class_name] = all_pixels_in_class

        return pixel_classes

def _main(args):
    model_path = os.path.expanduser(args.model_path)
    assert model_path.endswith('.h5'), 'Keras model must be a .h5 file.'
    anchors_path = os.path.expanduser(args.anchors_path)
    classes_path = os.path.expanduser(args.classes_path)
    test_path = os.path.expanduser(args.test_path)
    output_path = os.path.expanduser(args.output_path)

    if not os.path.exists(output_path):
        print('Creating output path {}'.format(output_path))
        os.mkdir(output_path)

    sess = K.get_session()  # TODO: Remove dependence on Tensorflow session.

    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    with open(anchors_path) as f:
        anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        anchors = np.array(anchors).reshape(-1, 2)

    yolo_model = load_model(model_path)

    # Verify model, anchors, and classes are compatible
    num_classes = len(class_names)
    num_anchors = len(anchors)
    # TODO: Assumes dim ordering is channel last
    model_output_channels = yolo_model.layers[-1].output_shape[-1]
    print('num_classes: {}    num_anchors= {}     modeloutputchannels: {}'.format(num_classes,num_anchors,model_output_channels))
    assert model_output_channels == num_anchors * (num_classes + 5), \
        'Mismatch between model and given anchor and class sizes. ' \
        'Specify matching anchors and classes with --anchors_path and ' \
        '--classes_path flags.'
    print('{} model, anchors, and classes loaded.'.format(model_path))

    # Check if model is fully convolutional, assuming channel last order.
    model_image_size = yolo_model.layers[0].input_shape[1:3]
    is_fixed_size = model_image_size != (None, None)

    # Generate colors for drawing bounding boxes.
    hsv_tuples = [(x / len(class_names), 1., 1.)
                  for x in range(len(class_names))]
    colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
    colors = list(
        map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
            colors))
    random.seed(10101)  # Fixed seed for consistent colors across runs.
    random.shuffle(colors)  # Shuffle colors to decorrelate adjacent classes.
    random.seed(None)  # Reset seed to default.

    # Generate output tensor targets for filtered bounding boxes.
    # TODO: Wrap these backend operations with Keras layers.
    yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))
    input_image_shape = K.placeholder(shape=(2, ))
    boxes, scores, classes = yolo_eval(
        yolo_outputs,
        input_image_shape,
        score_threshold=args.score_threshold,
        iou_threshold=args.iou_threshold)

    all_images_file = open('all_images_test.csv','w')

    for image_file in os.listdir(test_path):
        print('IMAGE_FILE: {}'.format(image_file))
        try:
            image_type = imghdr.what(os.path.join(test_path, image_file))
            if not image_type:
                continue
        except IsADirectoryError:
            continue

        image = Image.open(os.path.join(test_path, image_file))
        if is_fixed_size:  # TODO: When resizing we can use minibatch input.
            resized_image = image.resize(
                tuple(reversed(model_image_size)), Image.BICUBIC)
            image_data = np.array(resized_image, dtype='float32')
        else:
            # Due to skip connection + max pooling in YOLO_v2, inputs must have
            # width and height as multiples of 32.
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            resized_image = image.resize(new_image_size, Image.BICUBIC)
            image_data = np.array(resized_image, dtype='float32')

            print(image_data.shape)

        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = sess.run(
            [boxes, scores, classes],
            feed_dict={
                yolo_model.input: image_data,
                input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })
        print('Found {} boxes for {}'.format(len(out_boxes), image_file))

        font = ImageFont.truetype(
            font='font/FiraMono-Medium.otf',
            size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300

        # all boxes and classes for an image
        image_boxes = []


        for i, c in reversed(list(enumerate(out_classes))):

            predicted_class = class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            label = '{} {:.2f}'.format(predicted_class, score)

            draw = ImageDraw.Draw(image)
            label_size = draw.textsize(label, font)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
            print(label, (left, top), (right, bottom))

            image_boxes.append([label, (left, top), (right, bottom)])


            if top - label_size[1] >= 0:
                text_origin = np.array([left, top - label_size[1]])
            else:
                text_origin = np.array([left, top + 1])

            # My kingdom for a good redistributable image drawing library.
            for i in range(thickness):
                draw.rectangle(
                    [left + i, top + i, right - i, bottom - i],
                    outline=colors[c])
            draw.rectangle(
                [tuple(text_origin), tuple(text_origin + label_size)],
                fill=colors[c])
            draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            del draw

        image.save(os.path.join(output_path, image_file), quality=90)


        output_file = open('output_annotations/test_output_{}_.csv'.format(image_file), 'w')

        for class_name, class_values in get_pixels_for_each_class(image_boxes, image).items():

            if class_values:

                pixel_pairs = {}
                counter = class_values[0]
                current_distance = 0

                for i in range(len(class_values) -1 ):

                    if(class_values[i] + 1 == class_values[i+1]):
                        current_distance += 1
                        pixel_pairs[counter] = current_distance
                    else:
                        counter = class_values[i]
                        current_distance = 0

                final_array = []

                for key, value in pixel_pairs.items():
                    final_array.append(key)
                    final_array.append(value)

                positions_string = ""

                for item in final_array:
                    positions_string += str(item) + " "

                class_line = "{}_{}, {} \n".format(image_file, class_name, positions_string)
                print(class_name)
                #print(class_line)
                output_file.write(class_line)
                all_images_file.write(class_line)
            else:
                class_line = "{}_{},1 0 \n".format(image_file, class_name)
                #print(class_line)
                output_file.write(class_line)
                all_images_file.write(class_line)

        output_file.close()
    sess.close()

    all_images_file.close()


if __name__ == '__main__':
    _main(parser.parse_args())


ImportError: No module named 'yad2k'

A further issue we found when testing was that the network would not perform correctly when given a greyscale image. Therefore, we are required to resave each black and white image so our network can take it as input.
### Computational Resources
Training a neural network to classify and localise images requires a large amount of computational resource. We quickly learnt that it was not feasible to train on our laptops for two reasons. Firstly, the previously mentioned amount or resource required meant that it would take an infeasible time given the time constraints of this assignment, and secondly running from a laptop requires the laptop to be open and awake for processes to run. We decided to turn to the cloud to solve this problem. This allowed us to rent powerful virtual machines with professional GPU’s (rather than gaming GPU’s found in consumer desktop/ laptops). We also installed and configured [Jenkins](https://jenkins.io/) on our VM. Jenkins is a job orchestration tool that can be controlled via a web interface. This allowed us to define and schedule jobs on the neural network. As the jenkins user is running the tasks, we were able to not worry about maintaining an open session and could check the status of Job’s via the web UI.

We initially installed on Amazon Web Services (AWS) however there were $300 worth or free credits available for setting up a new account on Google Cloud (GC). AWS provides Amazon machine images (AMI’s) that are pre built and ready for machine learning, with many key tools, such as tensorflow and keras pre installed. This made setup on AWS very fast. However when we moved to GC we had to install display drivers and many tools before we could get started. 
Even when training on a powerful cloud instance with 80GB of RAM (training the network on all the images), RAM would max out within seconds and the process would get killed by the operating system. To work around this we broke the training data into smaller batches and trained on these. Jenkins made this easy as we were able to queue up many training jobs, and we were able to check the web UI anytime of day to get updated on progress.

During training we came across a bug in the the YAD2K code. When drawing the bounding boxes on images during the training cycle, it encountered a type mismatch bug, int -> float. Updating a single function argument fixed this issue and training jobs were able to then able to run successfully.

We aim to train on 2000 images 8 times and combine them into yolo.weights for testing.

### Performance Analysis:
We will discuss an accuracy measurement which is used to give the confidence of the network for a particular object. First, we introduce the notion of “Intersection over Union” (or IoU). While training our network, we can judge our results and analyse the accuracy because we have ground truth boxes. Below, we see that the green box is the position of object what we can get from training data (ground truth) and the red one is our predicted position after training.

<img src="files/figures/figure3_2.png">

This visual representation from Adrian Bosebrock (2016) demonstrates the usefulness of IOU measurements. In order to calculate this IOU value, we take the area of overlap divided by the area of union.

<img src="files/figures/figure3_3.png">

With this IOU value, we obtain the confidence by multiplying it by the probability that the object is of that certain class, ie:

$$ Confidence = Pr(Class_i) ∗ IOU^{truth}_{pred} $$

From the figure below, it can be seen that our network is 80% confident that the object in pink box is a truck. 

<img src="files/figures/figure3_4.png">

Our aim is to maximise this confidence.

## Experiments
Now that we have a functioning network that we can train, we turn to experimenting. Let’s recap what we will be experimenting with:
<ul>
    <li>Varying Activation Functions
    <li>Adjusting Anchor Box Sizes
    <li>Learning Rates
</ul>

### Varying Activation Functions:
Our exploration of the network began with the activation functions. While looking through the code of this neural network, we changed the activation functions of the network and ran some tests to see how it would affect the output. However, this clearly makes little logical sense as changing the activation layer will be altering the network which has been trained on a certain data set. Altering functions in a pre-trained network should have bad implications on the accuracy of the outputs.

The original activation functions were “leaky” (with gradient 0.1), with the exception of the penultimate layer where the activation function was “linear”. We changed these activation functions (and their derivatives) and tried implementing the following:
<ul>
    <li>Changed all activations to “linear”.
    <li>Changed to Relu activations.
    <li>Changed to leaky activation with varying gradients (0.0001, 0.05, 0.11, 0.105, 0.1075, 0.115, 0.2, 0.5, 0.8).
</ul>

These changes were completely at random and their purpose was simply to see what the network would do with these changes.

When we made these changes, we found that some interesting results. Unsurprisingly, many of these functions completely over classified in the wrong areas, resulting in completely ludicrous outputs. On the other hand, some of the outputs classified nothing at all. These outputs are to be expected because of the points stated above.

However, something unexpected happened when we used a leaky activation with gradient 0.11- only a slight variant on the original. On the particular image we were testing, the network managed to detect a motorbike in the top left hand side- an object that was not detected by the original trained network!

Although the bounding boxes on the originally detected objects were slightly a skew, it was a surprise to see that this network seemed better on detection. What seemed to happen is that the there was a decrease of the threshold of detection, allowing for more questionable objects to be detected that were in fact correct. However this was only tested on a single image and should be explored further before drawing any conclusions.

Below are the results.

Normal yolo.cfg:
<img src="files/figures/figure4_1.png">

Linear activations:
<img src="files/figures/figure4_2.png">

Relu activations and leaky activation with 0.0001 and 0.05 (just didn’t classify anything):
<img src="files/figures/figure4_3.png">

Leaky with 0.5:
<img src="files/figures/figure4_4.png">

Leaky with 0.8:
<img src="files/figures/figure4_5.png">

Leaky with 0.2:
<img src="files/figures/figure4_6.png">

Leaky 0.11:
<img src="files/figures/figure4_7.png">

Leaky 0.115:
<img src="files/figures/figure4_8.png">

Leaky 0.105:
<img src="files/figures/figure4_9.png">

Leaky 0.1075:
<img src="files/figures/figure4_10.png">

We can analogise changing the network after it has been trained to moving football goal posts after a ball has been kicked. With this, it is incredibly difficult or impossible to mathematically reason that your network is going in the right direction (or our goal posts will aline with the trajectory of the ball).

Unfortunately, due to time constraints, we were unable to explore this branch further and delve into the mathematics of why this happens. Regardless, it was an interesting aside to “move the network to optimize” instead of the traditional gradient descent approach.
### Varying Anchor Box Sizes:
A possible parameter we could experiment with is the number of anchor boxes and their dimensions. We tackle this by considering several approaches:
<ol>
    <li>The first could be just to pick varying sizes by hand. This method will come with it’s obvious disadvantages as it is not statistically rigorous. We omit this method.
    <li>An alternative is to use k-means algorithm to select the number of boxes. This is a possible option, however due to lack of time, implementing this algorithm may hinder progress so we choose to not use this. YOLOv2 uses 5 anchor boxes, and according to [Vivek Yadav](https://medium.com/@vivek.yadav/part-1-generating-anchor-boxes-for-yolo-like-network-for-vehicle-detection-using-kitti-dataset-b2fe033e5807), after some k-means experimentation, the conclusion is that 5 anchor boxes is the best for this network (note they are using a different dataset but the general idea is the same).
    <li>Use k-means algorithm to select the dimensions of the boxes. As we are choosing 5 anchor boxes, we could run 5-means algorithm to calculate the best dimensions for these anchor boxes. However, our data set is very similar to the data set that this network has already been trained on, so it would make sense to omit this experimentation as it has already been optimized to this factor and save some time.
</ol>

The 5 anchor box dimensions we will use are:

$$((0.57273, 0.677385),(1.87446, 2.06253),(3.33843, 5.47434),(7.88282, 3.52778),(9.77052, 9.16828))$$

### A Note on the Number of Epochs:
Initially, we thought that we could experiment with the number of epochs that our network runs through. However, our network implementation already uses “early stopping” which ensure that a network does not overfit to a certain dataset.

### Kaggle
After testing our data set with some pretrained weights, we received a score of 0.93377. This shows that the pretrained weights are already very well trained. This also gives us a control score so we could then see how we can improve on this.

## Conclusion

### Summary
We picked YAD2K as our platform to experiment with. It is a tensorflow / Keras implementation of YOLO. Yolo has some of the best result in image classification and localisation and there is a lot of literature about it, making it seem like an ideal choice, as mentioned before.

From inspecting the code there appear to be many easily accessible variables and parameters that could potentially be tweaked, providing a good basis for experimentation.

The most important reason for picking an existing network is time. Training a network from scratch would take at least 2 weeks, and that is after getting the network topology correct, designing the input and output, testing the codes, and implement a build platform. This endeavour would take on the order or months, and would require money to keep servers running.

We installed yad2k on google cloud VM, alongside a copy of Jenkins. This gives us the power to queue training jobs and allow them to run automatically.

With a pretrained network we can chop of the last layer and retrain the for the images and classes that we are working with. Retraining in this manner is a well documented practice, and allows flexibility in training on similar data.

We were able to train small batches of training data. However we were not able to chain together our training. Retraining takes in a Keras model. The model can be built with a script included with YAD2K. This script takes a set of weights and a config file. The config file describes the shape of the network.  After completing a training cycle YAD2K provides a new set of weights. We then needed to pass these weights through the script with the config file to generate a new Keras model. This would allow us to train on subsequent data and continue to build and improve out weights.

However, we have not be able to successfully build new models from our weights. We suspected that we might need a new config to build our initial model from. We download the VOC config and VOC weights developed by yolo as these match the results we are looking for. However the YAD2K script fails to build a Keras model, the works successfully. We do get a model file, however the loss at each step in the training is 'NAN' (not a number).

From our experiments and implementations, we have run into many barriers with YAD2K and unfortunately have not produced the most tangible results.

NB: We are still continuing to try and train a model for our data on our platform but have no results at the time of submission. This training will need to move off of Google Cloud once our credits have run out.

Our findings could have been very much improved if we had more time and resources. Also, more computational power would have been desirable so we could train these networks to our own training data, from scratch.

We could also designed our own datasets consisting of simple data, such as coloured shapes, to see how well our network generalises and how fast it does so.

After our testing, we obtained some results. An example of one is below.

<img src="files/figures/figure5_1.jpg">

In this picture, we can see objects can be detected with high confidence such as one chair got 0.84.
Furthermore, we also output the pixels of all objects like the format of training data set:

<code>2008_000043.jpg_aeroplane,1 0 
2008_000043.jpg_bicycle,1 0 
2008_000043.jpg_bird,1 0 
2008_000043.jpg_boat,1 0 
2008_000043.jpg_bottle,1 0 
2008_000043.jpg_bus,1 0 
2008_000043.jpg_car,1 0 
2008_000043.jpg_cat,1 0 
2008_000043.jpg_chair, 65322 32 65354 32 65754 32 66154 32 66554 32 66954 32 67354 32 67754 32 68154 32 68554 41 68649 32 68954 41 69049 45 69106 32 69354 41 69449 45 69506 32 69754 41 69849 45 69906 32 70154 41 70249 45 70306 32 70554 41 70649 45 70706 32 70954 41 71049 45 71106 32 71354 41 71449 45 71506 32 71754 41 71849 45 71906 32 72154 41 72249 45 72306 32 72554 41 72649 45 72706 32 72954 41 73049 45 73106 32 73354 41 73449 45 73506 32 73754 41 73849 45 73906 32 74154 41 74249 45 74306 32 74554 41 74649 45 74706 32 74954 41 75049 45 75106 32 75354 41 75449 45 75506 32 75754 41 75849 45 75906 32 76154 41 76249 45 76306 32 76554 41 76649 45 76706 32 76954 41 77049 45 77106 32 77354 41 77449 45 77506 32 77754 41 77849 45 77906 32 78154 41 78249 89 78350 32 78554 41 78649 89 78750 32 78954 41 79049 89 79150 32 79354 89 79550 32 79754 89 79950 32 80154 89 80350 32 80554 89 80750 32 80954 89 81150 32 81354 46 81550 87 81809 46 81950 87 82209 46 82350 87 82609 74 82702 46 82750 87 83009 74 83102 46 83150 87 83409 74 83502 46 83550 87 83809 74 83902 46 83950 87 84209 74 84302 46 84350 87 84609 74 84702 46 84750 87 85009 74 85102 46 85150 87 85409 74 85502 46 85550 87 85809 74 85902 46 85950 87 86209 74 86302 46 86350 87 86609 74 86702 46 86750 87 87009 74 87102 46 87150 87 87409 74 87502 46 87550 87 87809 74 87902 46 87950 87 88209 74 88302 46 88350 87 88609 74 88702 46 88750 87 89009 74 89102 46 89150 87 89409 74 89502 46 89550 87 89809 74 89902 46 89950 87 90209 74 90302 46 90350 87 90609 74 90702 46 90750 87 91009 74 91102 46 91150 87 91409 74 91502 46 91550 87 91809 74 91902 46 91950 87 92209 74 92302 46 92350 87 92609 74 92702 46 92750 87 93009 74 93102 46 93150 87 93409 74 93502 46 93550 87 93809 74 93902 46 93950 87 94209 74 94302 46 94350 87 94609 74 94702 46 94750 87 95009 74 95102 46 95150 87 95409 74 95502 46 95550 87 95809 74 95902 46 95950 87 96209 74 96302 46 96350 87 96609 74 96702 46 96750 87 97009 74 97102 46 97150 87 97409 74 97502 46 97550 87 97809 74 97902 46 97950 87 98209 74 98302 46 98350 87 98609 74 98702 46 98750 87 99009 74 99102 46 99150 87 99409 74 99502 46 99550 87 99809 74 99902 46 99950 87 100209 74 100302 46 100350 87 100609 74 100702 46 100750 87 101009 74 101102 46 101150 87 101409 74 101502 46 101550 87 101809 74 101902 46 101950 87 102209 74 102302 46 102350 87 102609 74 102702 46 102750 87 103009 74 103102 46 103150 87 103409 74 103502 46 103550 87 103809 74 103902 46 103950 87 104209 74 104302 46 104350 87 104609 74 104702 46 104750 87 105009 74 105102 46 105150 87 105409 74 105502 46 105550 87 105809 74 105902 46 105950 87 106209 74 106302 46 106350 87 106609 74 106702 46 106750 87 107009 74 107102 46 107150 87 107409 74 107502 46 107550 87 107809 74 107902 46 107950 87 108209 74 108302 46 108350 87 108609 74 108702 46 108750 87 109009 74 109102 46 109150 87 109409 74 109502 46 109550 87 109809 74 109902 46 109950 87 110209 74 110302 46 110350 87 110609 74 110702 46 110750 87 111009 74 111102 46 111150 87 111409 74 111502 46 111550 87 111809 74 111902 46 111950 87 112209 74 112302 46 112350 87 112609 74 112702 46 112750 87 113009 74 113102 46 113150 87 113409 74 113502 46 113550 87 113809 74 113902 46 113950 87 114209 74 114302 46 114350 87 114609 74 114702 46 114750 87 115009 74 115102 46 115150 87 115409 74 115502 46 115550 87 115809 74 115902 46 115950 87 116209 74 116302 46 116350 87 116609 74 116702 46 116750 87 117009 74 117102 46 117150 87 117409 74 117502 46 117550 87 117809 74 117902 46 117950 87 118209 74 118302 46 118350 87 118609 74 118702 46 118750 87 119009 74 119102 46 119150 87 119409 74 119502 46 119550 87 119809 74 119902 46 119950 87 120209 74 120302 46 120350 87 120609 74 120702 46 120750 87 121009 74 121102 46 121150 87 121409 74 121502 46 121550 87 121809 74 121902 46 121950 87 122209 74 122302 46 122350 87 122609 74 122702 46 122750 87 123009 74 123102 46 123150 87 123409 74 123502 46 123550 87 123809 74 123902 46 123950 87 124209 74 124302 46 124350 87 124609 74 124702 46 124750 87 125009 74 125102 46 125150 87 125409 74 125502 46 125550 87 125809 74 125902 46 125950 87 126209 74 126302 46 126350 87 126609 74 126702 46 126750 87 127009 74 127102 46 127150 75 127409 74 127502 46 127550 75 127809 74 127902 46 127950 75 128209 74 128302 46 128350 75 128609 74 128702 46 128750 75 129009 74 129102 46 129150 75 129409 74 129502 46 129550 75 129809 74 129902 46 129950 75 130209 74 130302 46 130350 75 130609 74 130702 46 130750 75 131009 74 131102 46 131150 75 131409 74 131502 46 131550 75 131809 74 131902 46 131950 75 132209 74 132302 46 132350 75 132609 74 132702 46 132750 75 133009 74 133102 46 133150 75 133409 74 133502 46 133550 75 133809 74 133902 46 133950 75 134209 74 134302 46 134350 75 134609 74 134702 46 134750 75 135009 74 135102 46 135150 75 135409 74 135502 46 135550 75 135809 74 135902 46 135950 75 136209 74 136302 46 136350 75 136609 74 136702 46 136750 75 137009 74 137102 75 137409 74 137502 75 137809 74 137902 75 138209 74 138302 75 138609 74 138702 75 139009 74 139102 75 139409 74 139502 75 139809 74 139902 75 140209 74 140302 75 140609 74 140702 75 141009 74 141102 75 141409 74 141502 75 141809 74 141902 75 142209 74 142302 75 142609 74 142702 75 143009 74 143102 75 143409 74 143502 75 143809 74 143902 75 144209 74 144302 75 144609 74 144702 75 145009 74 145102 75 145409 74 145502 75 145809 74 145902 75 146209 74 146302 75 146609 74 146702 75 147009 74 147102 75  
2008_000043.jpg_cow,1 0 
2008_000043.jpg_diningtable, 75738 203 75941 203 76341 203 76741 203 77141 203 77541 203 77941 203 78341 203 78741 203 79141 203 79541 203 79941 203 80341 203 80741 203 81141 203 81541 203 81941 203 82341 203 82741 203 83141 203 83541 203 83941 203 84341 203 84741 203 85141 203 85541 203 85941 203 86341 203 86741 203 87141 203 87541 203 87941 203 88341 203 88741 203 89141 203 89541 203 89941 203 90341 203 90741 203 91141 203 91541 203 91941 203 92341 203 92741 203 93141 203 93541 203 93941 203 94341 203 94741 203 95141 203 95541 203 95941 203 96341 203 96741 203 97141 203 97541 203 97941 203 98341 203 98741 203 99141 203 99541 203 99941 203 100341 203 100741 203 101141 203 101541 203 101941 203 102341 203 102741 203 103141 203 103541 203 103941 203 104341 203 104741 203 105141 203 105541 203 105941 203 106341 203 106741 203 107141 203 107541 203 107941 203 108341 203 108741 203 109141 203 109541 203 109941 203 110341 203 110741 203 111141 203 111541 203 111941 203 112341 203 112741 203 113141 203 113541 203 113941 203 114341 203 114741 203 115141 203 115541 203 115941 203 116341 203 116741 203 117141 203 117541 203 117941 203 118341 203 118741 203 119141 203 119541 203 119941 203 120341 203 120741 203 121141 203 121541 203 121941 203 122341 203 122741 203 123141 203 123541 203 123941 203 124341 203 124741 203 125141 203 125541 203 125941 203 126341 203 126741 203 127141 203 127541 203 127941 203 128341 203 128741 203 129141 203 129541 203 129941 203 130341 203 130741 203 131141 203 131541 203 131941 203 132341 203 132741 203 133141 203 133541 203 133941 203 134341 203 134741 203 135141 203 135541 203 135941 203 136341 203  
2008_000043.jpg_dog,1 0 
2008_000043.jpg_horse,1 0 
2008_000043.jpg_motorbike,1 0 
2008_000043.jpg_person,1 0 
2008_000043.jpg_pottedplant, 59977 20 59997 20 60397 20 60797 20 61197 20 61597 20 61997 20 62397 20 62797 20 63197 20 63597 20 63997 20 64397 20 64797 20 65197 20 65597 20 65997 22 66399 22 66799 22 67199 22 67599 22 67999 22 68399 22 68799 22 69199 22 69599 22 69999 22 70399 22 70799 22 71199 22 71599 22 71999 22 72399 22 72799 22 73199 22 73599 22 73999 22 74399 22 74799 22 75199 22 75599 22 75999 22 76399 22 76799 22 77199 22 77599 22 77999 22 78399 22 78799 22 79199 22 79599 22 79999 22 80399 22 80799 22 81199 22 81599 22 81999 22 82399 22 82799 22 83199 22 83599 22 83999 22 84399 22 84799 22 85199 22 85599 22 85999 22 86399 22 86799 22 87199 22 87599 22 87999 22 88399 22 88799 22 89199 22 89599 22 89999 22 90399 22 90799 22 91199 22 91599 22 91999 22 92399 22 92799 22 93199 22 93599 22 93999 22 94399 22 94799 22 95199 22 95599 22 95999 22 96399 22 96799 22 97199 22 97599 22 97999 22 98399 22 98799 22 99199 22 99599 22 99999 22 100399 22 100799 22 101199 22 101599 22 101999 22 102399 22 102799 22 103199 22 103599 22 103999 22 104399 22 104799 22 105199 22 105599 22 105999 22 106399 22 106799 22 107199 22 107599 22 107999 22 108399 22 108799 22 109199 22 109599 20 109997 20 110397 20 110797 20 111197 20 111597 20 111997 20 112397 20 112797 20 113197 20 113597 20 113997 20 114397 20 114797 20 115197 20 115597 20 115997 20 116397 20 116797 20  
2008_000043.jpg_sheep,1 0 
2008_000043.jpg_sofa,1 0 
2008_000043.jpg_train,1 0 
2008_000043.jpg_tvmonitor,1 0 
</code>

Overall, we have gained valuable and practical experience in tensorflow and keras. Configuring a practical testing environment for group work and large compute load has also been great!

## Description of contribution

Jon- Set up the testing platforms and modified the relevant code.
Ed- Structured the report and calculated mathematical formulae.
Yawen- Set up google cloud platform and researched relevant background knowledge.
Shiyu- Researched relevant background knowledge.